##  zookeeper菜谱—— 用zookeeper实现分布式栅栏，队列，锁，以及两阶段提交，leader选取。

### 分布式栅栏
> 分布式系统使用栅栏来完成一系列节点的操作等待一个条件的行为。他会阻塞所有相关节点的操作，知道该条件满足。
> zookeeper通过栅栏节点来实现栅栏的功能。栅栏节点存在，则所有操作被阻塞，实现栅栏的原理如下：
> * 客户点调用zookeeper的exists()接口，并设置监听为true

> * 如果exists()接口返回为false，那么说明栅栏节点以及消失，客户端继续

> * 如果exists()接口返回为true，那么说明栅栏存在，客户端会保持等待zookeeper的watcher通知事件。

> * 当监听被触发，客户端重新调用exists()接口，继续以上几个步骤，直到栅栏节点被移除。


> 双栅栏
> 双栅栏允许客户端在操作前后都进行同步（所有线程到达时，开始栅栏中的操作，一个线程在栅栏中操作完成时，可以离开栅栏）。当全部线程（x个）完成指定操作到达栅栏，线程开始操作，并在操作后离开栅栏。
> 以下伪码，将栅栏节点命名为b，每个客户端操作p在开始时，在b上注册，并在操作结束时，离开p。一个节点经过以下程序来完成在栅栏节点的注册，他会阻塞> 操作，直到客户端x在操作前完成在栅栏上的注册（这里的x及其操作取决于你的系统需要）
> * 进入栅栏：

>   1. 创建一个名字： b+“/”+p
>  1. 设置监听  exists(b + ‘‘/ready’’, true) ——在b/ready节点上设置监听
>   1. 新建临时子节点create( n, EPHEMERAL) ——在栅栏节点下新建临时子节点b/p
>   1. L = getChildren(b, false) ——  获取栅栏节点下的子节点（所有阻塞的客户端节点）
>  1. 如果L的子节点比x个数少，则等待监听事件 ——继续监听
>   1. 否则 create(b + ‘‘/ready’’, REGULAR) ——触发所有客户端在b/ready上的监听


> * 离开栅栏（优先级最低的线程作为flag）：

>   1. L = getChildren(b, false)
>   1. 如果没有子节点，则退出
>   1. 如果p是唯一子节点，那么删除n后退出
>   1. 如果p是最低优先级的节点，那么等待高优先级节点到达n
>   1. 否则，若p存在，且仍然有其他操作未到达栅栏，则删除p的节点 b/p
>   1. 回到步骤1



### 分布式队列
> 分布式队列时分布式系统中常用的数据结构。用zookeeper实现一个分布式队列的方式如下：
> * 首先设置一个znode来保存队列，作为队列节点。
> * 入队操作：客户端调用create()方法，并且路径名以“queue-”结尾，并且设置该节点的CreateMode为：ephemeral——true，sequence ——true。这样，新建节点的命名形式为：_path-to-queue-node_/queue-X，其中X是自增的。
> * 出队操作：调用getChildren( )，并且设置监听队列节点为true。并且移除X最小的节点（最先入队）。客户端只需要在全部本地查询到的队列元素处理完毕，或者监听触发时（我自己加的）。再去重新调用getChildren（）。如果队列节点下不存在任何子节点，客户端将等待监听触发时，再一次查询队列元素。




### 分布式锁
> 完全的分布式锁，即在任意时刻，没有两个客户端可以持有相同的锁。zookeeper的实现方式如下：
> * 设置一个锁节点
> * 获取锁
>   1. 客户端调用create()方法，路径命名：“_locknode_/lock-”，并且设置该节点的CreateMode为：ephemeral——true，ephemeral——true。
>   1. 在锁节点上调用getChildren( )，并且不设置监听（以避免 herd effect）
>   1. 如果在获取的子节点中，步骤1创建的节点的序列号是最小的，则客户端已经获取了这个锁（最先创建节点。）
>   1. 如果没有获得锁，则调用在比当前节点序列号小1的节点上，调用exist方法
>   1. 如果返回false，则继续第2步骤，否则等待该节点上的监听时间后，在继续第2步骤

> * 释放锁
>   + 删除步骤1中创建的节点

> * 独占锁的一些注意点
>   1. 一个节点的移除只会导致一个等待锁的客户端节点的监听事件被触发。这样可以避免集群效应。
>   1. 过程中，没有投票以及超时设置
>   1. 这种锁的实现方式，很容易获取锁上竞争的个数，中断锁，在锁上进行debug。

> zookeeper也可以实现共享锁，如读写锁：
> * 设置锁节点
> * 获取读锁
>   1. 客户端调用create()方法，创建节点，path命名为："_locknode_/read-"，并且设置该节点的CreateMode为：ephemeral——true，ephemeral——true。
>   1. 在锁节点上调用getChildren（），不设置监听（避免herd effect）
>   1. 如果不存在"write-"开头的，且比步骤1创造的节点序号更小子节点（在读之前有写操作），则客户端获取锁，退出此流程
>   1. 否则，调用exists()方法，设置比当前读节点序号小的write节点里，最大节点的监听
>   1. 如果exists返回false，则回到步骤2
>   1. 否则，收到监听事件通知后回到步骤2

> * 获取写锁
>  1. 客户端调用create()方法，创建节点，path命名为："_locknode_/write-"，并且设置该节点的CreateMode为：ephemeral——true，ephemeral——true。调用getData( )，并设置监听。
>  1. 在锁节点上调用getChildren（），不设置监听（避免herd effect）
>  1. 如果不存在更小序列号的子节点，则获取锁，退出该流程
>  1. 否则，在次低的节点上调用exists()，并设置监听
>  1. 如果方法返回false，回到步骤2，
>  1. 否则，等待监听通知事件发生后，回到步骤2.

> * 读写锁的一些注意点
>  + 可能会造成集群效应。如 ，在大量读锁等待一个写锁时。 

> zookeeper也可以实现可恢复共享锁：

> 待实现
